# Graph Theory
## Exercise List 7

In [47]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from graphframes import *

In [2]:
conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)
vertexes = spark.read.csv('pe_cities.csv', header=True)
edges = spark.read.csv('pe_cities_distances.csv', header=True)
graph = GraphFrame(vertexes, edges)

In [19]:
graph_degrees = graph.degrees
graph_degrees.sort("degree", ascending=False).show()

+--------------------+------+
|                  id|degree|
+--------------------+------+
|Brejo da Madre de...|     7|
|            Igarassu|     7|
|           Araripina|     6|
|                 Exu|     6|
|         Tupanatinga|     6|
|              Buíque|     6|
|        João Alfredo|     6|
|            Ouricuri|     6|
|               Ipubi|     6|
|              Bodocó|     4|
+--------------------+------+



- Calculate the number of triangles

In [11]:
result = graph.triangleCount()
result.sort("count", ascending=False).show()

+-----+--------------------+---------+----------+----------+
|count|                  id| latitude| longitude|population|
+-----+--------------------+---------+----------+----------+
|   12|Brejo da Madre de...|-8.145833|-36.370833|     50752|
|   12|            Igarassu|-7.834167|-34.906389|    117019|
|   10|              Buíque|-8.623333|-37.156389|     58378|
|   10|         Tupanatinga|-8.752778|    -37.34|     27304|
|   10|               Ipubi|-7.651944|-40.148889|     30854|
|    9|           Araripina|-7.575833|-40.497778|     84418|
|    9|            Ouricuri|  -7.8825|-40.081667|     69459|
|    9|                 Exu|-7.511944|-39.723889|     31825|
|    8|        João Alfredo|-7.855833|-35.587778|     33822|
|    4|              Bodocó|-7.778333|-39.941111|     38146|
+-----+--------------------+---------+----------+----------+



- Calculate the local clustering coefficient of the three highest degree vertexes

In [41]:
triangles_ordered = result.sort("count", ascending=False)
degrees_ordered = graph_degrees.sort("degree", ascending=False)
triangles = triangles_ordered.select("id", "count").collect()
degrees = degrees_ordered.select("id", "degree").collect()
output = dict(city_id=list(), localClusterCoefficient=list())
i = 0
for triangle_value in triangles:
    for degree_value in degrees:
        if (triangle_value.id == degree_value.id):
            lccf = (2 * triangle_value['count']) / (degree_value['degree'] * (degree_value['degree'] - 1))
            output['localClusterCoefficient'].append(lccf)
            output['city_id'].append(degree_value.id)
            i += 1
    if (i == 3):
        break

print(output)

{'city_id': ['Igarassu', 'Brejo da Madre de Deus', 'Buíque'], 'localClusterCoefficient': [0.5714285714285714, 0.5714285714285714, 0.6666666666666666]}


- Determine the strongly connected components and the connected components. How much do they differ?

In [42]:
scc = graph.stronglyConnectedComponents(maxIter=10)
scc.show()

+--------------------+---------+----------+----------+------------+
|                  id| latitude| longitude|population|   component|
+--------------------+---------+----------+----------+------------+
|Brejo da Madre de...|-8.145833|-36.370833|     50752|154618822656|
|              Bodocó|-7.778333|-39.941111|     38146|154618822656|
|        João Alfredo|-7.855833|-35.587778|     33822|154618822656|
|               Ipubi|-7.651944|-40.148889|     30854|154618822656|
|           Araripina|-7.575833|-40.497778|     84418|154618822656|
|            Igarassu|-7.834167|-34.906389|    117019|154618822656|
|                 Exu|-7.511944|-39.723889|     31825|154618822656|
|              Buíque|-8.623333|-37.156389|     58378|154618822656|
|         Tupanatinga|-8.752778|    -37.34|     27304|154618822656|
|            Ouricuri|  -7.8825|-40.081667|     69459|154618822656|
+--------------------+---------+----------+----------+------------+



In [50]:
sc.setCheckpointDir('./checkpoints')
ccp = graph.connectedComponents(checkpointInterval=2, broadcastThreshold=10000000)
ccp.show()

+--------------------+---------+----------+----------+------------+
|                  id| latitude| longitude|population|   component|
+--------------------+---------+----------+----------+------------+
|            Igarassu|-7.834167|-34.906389|    117019|154618822656|
|           Araripina|-7.575833|-40.497778|     84418|154618822656|
|            Ouricuri|  -7.8825|-40.081667|     69459|154618822656|
|              Buíque|-8.623333|-37.156389|     58378|154618822656|
|Brejo da Madre de...|-8.145833|-36.370833|     50752|154618822656|
|              Bodocó|-7.778333|-39.941111|     38146|154618822656|
|        João Alfredo|-7.855833|-35.587778|     33822|154618822656|
|                 Exu|-7.511944|-39.723889|     31825|154618822656|
|               Ipubi|-7.651944|-40.148889|     30854|154618822656|
|         Tupanatinga|-8.752778|    -37.34|     27304|154618822656|
+--------------------+---------+----------+----------+------------+



By the results above, there was no difference. 

- Determine the graph clusters by the execution of the label propagation algorithm

In [51]:
lpa = graph.labelPropagation(maxIter=10)
lpa.show()

+--------------------+---------+----------+----------+------------+
|                  id| latitude| longitude|population|       label|
+--------------------+---------+----------+----------+------------+
|Brejo da Madre de...|-8.145833|-36.370833|     50752|515396075520|
|              Bodocó|-7.778333|-39.941111|     38146|515396075520|
|        João Alfredo|-7.855833|-35.587778|     33822|515396075520|
|               Ipubi|-7.651944|-40.148889|     30854|515396075520|
|           Araripina|-7.575833|-40.497778|     84418|515396075520|
|            Igarassu|-7.834167|-34.906389|    117019|515396075520|
|                 Exu|-7.511944|-39.723889|     31825|515396075520|
|              Buíque|-8.623333|-37.156389|     58378|515396075520|
|         Tupanatinga|-8.752778|    -37.34|     27304|515396075520|
|            Ouricuri|  -7.8825|-40.081667|     69459|515396075520|
+--------------------+---------+----------+----------+------------+

